Py City: Observable Trends

1) Performance from charter schools tends to skew higher than district schools.  Average math and average reading scores are higher in charters, even though overall per student budget is lower  

2) Charter schools tend to have fewer students, which also highlights a potential self-selection bias in the data; i.e. certain students, based on variables such as neighborhood, income, etc. are probably more likely to attend charters vs district schools who are likely to have a more diverse population.  


In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [3]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
#calculate the total number of schools
total_schools = len(school_data)

In [5]:
#calculate the total number of students
student_total = len(student_data)

In [6]:
#calculate the total budget
total_budget = round(school_data["budget"]).sum()

In [7]:
#calculate the average math score
avg_math = school_data_complete["math_score"].mean()

In [8]:
#calculate the average reading score
avg_reading = school_data_complete["reading_score"].mean()

In [9]:
#calculate the overall passing rate
passing = (avg_math + avg_reading)/2

In [10]:
#calculate the percentage of students with a passing math score (>=70)
passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]
passing_math_percent = len(passing_math)/len(student_data)

In [11]:
#calculate the percentage of students with a passing reading score (>=70)
passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
passing_reading_percent = len(passing_reading)/len(student_data)

In [12]:
#District summary data frame
summary_df = pd.DataFrame({"Total Number of Schools": [total_schools],
                           "Total Number of Students": [student_total],
                           "Total Budget": [total_budget],
                          "Average Math Score": [avg_math],
                          "Average Reading Score": [avg_reading],
                          "Overall Pass Rate": [passing], 
                          "Percent of Students with Passing Math Score": [passing_math_percent],
                          "Percent of Students with Passing Reading Score": [passing_reading_percent]})
summary_df

,Total Number of Schools,Total Number of Students,Total Budget,Average Math Score,Average Reading Score,Overall Pass Rate,Percent of Students with Passing Math Score,Percent of Students with Passing Reading Score
0,15,39170,24649428,78.985371,81.87784,80.431606,0.749809,0.858055


In [13]:
#school summary: calculate average math and reading scores
school_group_df = school_data_complete.groupby(['school_name'])
student_count = school_group_df["Student ID"].count()
school_math = school_group_df["math_score"].mean()
school_reading = school_group_df["reading_score"].mean()

In [14]:
#school summary: calculate % passing math 
school_math_group_df = passing_math.groupby(['school_name'])
school_math_count = school_math_group_df["Student ID"].count()
passing_math = school_math_count / student_count

In [15]:
#school summary: calculate % passing reading 
school_reading_group_df = passing_reading.groupby(['school_name'])
school_reading_count = school_reading_group_df["Student ID"].count()
passing_reading = school_reading_count / student_count

In [16]:
#school summary: calculate overall passing rate
overall_pass = (passing_math + passing_reading) / 2

In [17]:
#calculate per student budget
school_data["Per Student Budget"] = school_data['budget'] / school_data['size']
school_data["Per Student Budget"] = school_data["Per Student Budget"].astype(float).map("${:,.2f}".format)
school_data["Budget"] = school_data["budget"].astype(float).map("${:,.2f}".format)

In [18]:
#create initial school summary table
school_summary_table = pd.DataFrame({"Student Count": student_count,
                                     "Average Math Score": school_math,
                                    "Average Reading Score": school_reading,
                                   "% Passing Math": passing_math,
                                   "% Passing Reading": passing_reading,
                                   "Overall Passing Rate": overall_pass})

In [19]:
#create final school summary table
joined_school_summary = pd.merge(school_summary_table, school_data, how="left", on=["school_name", "school_name"])
del joined_school_summary["School ID"]
del joined_school_summary["size"]
del joined_school_summary["budget"]

joined_school_summary = joined_school_summary.rename(columns={"school_name": "School Name", "type": "School Type"})
joined_school_summary = joined_school_summary[["School Name", "Student Count", "Budget", "Per Student Budget", "School Type"
                                              , "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]

joined_school_summary.head(20)

,School Name,Student Count,Budget,Per Student Budget,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,4976,"$3,124,928.00",$628.00,District,77.048432,81.033963,0.666801,0.819333,0.743067
1,Cabrera High School,1858,"$1,081,356.00",$582.00,Charter,83.061895,83.975780,0.941335,0.970398,0.955867
2,Figueroa High School,2949,"$1,884,411.00",$639.00,District,76.711767,81.158020,0.659885,0.807392,0.733639
3,Ford High School,2739,"$1,763,916.00",$644.00,District,77.102592,80.746258,0.683096,0.792990,0.738043
4,Griffin High School,1468,"$917,500.00",$625.00,Charter,83.351499,83.816757,0.933924,0.971390,0.952657
5,Hernandez High School,4635,"$3,022,020.00",$652.00,District,77.289752,80.934412,0.667530,0.808630,0.738080
6,Holden High School,427,"$248,087.00",$581.00,Charter,83.803279,83.814988,0.925059,0.962529,0.943794
7,Huang High School,2917,"$1,910,635.00",$655.00,District,76.629414,81.182722,0.656839,0.813164,0.735002
8,Johnson High School,4761,"$3,094,650.00",$650.00,District,77.072464,80.966394,0.660576,0.812224,0.736400
9,Pena High School,962,"$585,858.00",$609.00,Charter,83.839917,84.044699,0.945946,0.959459,0.952703


In [20]:
#sort schools by highest overall passing rate
joined_school_summary = joined_school_summary.sort_values("Overall Passing Rate", ascending = False)
joined_school_summary.head(5)

,School Name,Student Count,Budget,Per Student Budget,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
1,Cabrera High School,1858,"$1,081,356.00",$582.00,Charter,83.061895,83.975780,0.941335,0.970398,0.955867
12,Thomas High School,1635,"$1,043,130.00",$638.00,Charter,83.418349,83.848930,0.932722,0.973089,0.952905
9,Pena High School,962,"$585,858.00",$609.00,Charter,83.839917,84.044699,0.945946,0.959459,0.952703
4,Griffin High School,1468,"$917,500.00",$625.00,Charter,83.351499,83.816757,0.933924,0.971390,0.952657
13,Wilson High School,2283,"$1,319,574.00",$578.00,Charter,83.274201,83.989488,0.938677,0.965396,0.952037


In [21]:
#sort schools by lowest overall passing rate
joined_school_summary = joined_school_summary.sort_values("Overall Passing Rate", ascending = True)
joined_school_summary.head(5)

,School Name,Student Count,Budget,Per Student Budget,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
10,Rodriguez High School,3999,"$2,547,363.00",$637.00,District,76.842711,80.744686,0.663666,0.802201,0.732933
2,Figueroa High School,2949,"$1,884,411.00",$639.00,District,76.711767,81.158020,0.659885,0.807392,0.733639
7,Huang High School,2917,"$1,910,635.00",$655.00,District,76.629414,81.182722,0.656839,0.813164,0.735002
8,Johnson High School,4761,"$3,094,650.00",$650.00,District,77.072464,80.966394,0.660576,0.812224,0.736400
3,Ford High School,2739,"$1,763,916.00",$644.00,District,77.102592,80.746258,0.683096,0.792990,0.738043


In [22]:
#create panda series for each grade 
ninth_grade = school_data_complete.loc[school_data_complete["grade"] == "9th"]
ninth_grade = ninth_grade[["school_name", "math_score", "reading_score"]]

tenth_grade = school_data_complete.loc[school_data_complete["grade"] == "10th"]
tenth_grade = tenth_grade[["school_name", "math_score", "reading_score"]]

eleventh_grade = school_data_complete.loc[school_data_complete["grade"] == "11th"]
eleventh_grade = eleventh_grade[["school_name", "math_score", "reading_score"]]

twelth_grade = school_data_complete.loc[school_data_complete["grade"] == "12th"]
twelth_grade = twelth_grade[["school_name", "math_score", "reading_score"]]

In [23]:
#calcuate average scores for each grade
ninth_grade_group = ninth_grade.groupby(['school_name'])
avg_math_9th = ninth_grade_group["math_score"].mean()
avg_read_9th = ninth_grade_group["reading_score"].mean()

tenth_grade_group = tenth_grade.groupby(['school_name'])
avg_math_10th = tenth_grade_group["math_score"].mean()
avg_read_10th = tenth_grade_group["reading_score"].mean()

eleventh_grade_group = eleventh_grade.groupby(['school_name'])
avg_math_11th = eleventh_grade_group["math_score"].mean()
avg_read_11th = eleventh_grade_group["reading_score"].mean()

twelth_grade_group = twelth_grade.groupby(['school_name'])
avg_math_12th = twelth_grade_group["math_score"].mean()
avg_read_12th = twelth_grade_group["reading_score"].mean()

In [24]:
#final math scores by grade
grade_summary_table = pd.DataFrame({"9th": avg_math_9th,
                                   "10th": avg_math_10th,
                                   "11th": avg_math_11th,
                                   "12th": avg_math_12th})
grade_summary_table.head(20)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [25]:
#final reading scores by grade
grade_summary_table = pd.DataFrame({"9th": avg_read_9th,
                                   "10th": avg_read_10th,
                                   "11th": avg_read_11th,
                                   "12th": avg_read_12th})
grade_summary_table.head(20)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [26]:
school_data["Budget Bins"] = school_data['budget'] / school_data['size']
school_data['Budget Bins'] = pd.to_numeric(school_data['Budget Bins'])
school_data.dtypes

School ID               int64
school_name            object
type                   object
size                    int64
budget                  int64
Per Student Budget     object
Budget                 object
Budget Bins           float64
dtype: object

In [27]:
# Create Sample Bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["$585", "$585-615", "$615-645", "$645-675"]
school_data["Spending Summary"] = pd.cut(school_data["Budget Bins"], spending_bins, labels=group_names)

size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_data["School Size"] = pd.cut(school_data["size"], size_bins, labels=group_names)

bin_join = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

bin_join.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Per Student Budget,Budget,Budget Bins,Spending Summary,School Size
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,$655.00,"$1,910,635.00",655.0,$645-675,Large (2000-5000)
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,$655.00,"$1,910,635.00",655.0,$645-675,Large (2000-5000)
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,$655.00,"$1,910,635.00",655.0,$645-675,Large (2000-5000)
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,$655.00,"$1,910,635.00",655.0,$645-675,Large (2000-5000)
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,$655.00,"$1,910,635.00",655.0,$645-675,Large (2000-5000)


In [62]:
spend_gr_df = bin_join.groupby(['Spending Summary'])
spend_stu_count = spend_gr_df["Student ID"].count()
spend_math = spend_gr_df["math_score"].mean()
spend_read = spend_gr_df["reading_score"].mean()

size_gr_df = bin_join.groupby(['School Size'])
size_stu_count = size_gr_df["Student ID"].count()
size_math = size_gr_df["math_score"].mean()
size_read = size_gr_df["reading_score"].mean()

type_gr_df = bin_join.groupby(['type'])
type_stu_count = type_gr_df["Student ID"].count()
type_math = type_gr_df["math_score"].mean()
type_read = type_gr_df["reading_score"].mean()

In [63]:
math_pass = bin_join.loc[school_data_complete["math_score"] >= 70]
read_pass = bin_join.loc[school_data_complete["reading_score"] >= 70]

In [69]:
spend_math_df = math_pass.groupby(['Spending Summary'])
spend_read_df = read_pass.groupby(['Spending Summary'])
spend_math_stu_count = spend_math_df["Student ID"].count()
spend_read_stu_count = spend_read_df["Student ID"].count()
spend_percent_passing_math = spend_math_stu_count/spend_stu_count
spend_percent_passing_reading = spend_read_stu_count/spend_stu_count
spend_overall_passing_rate = (spend_percent_passing_math + spend_percent_passing_reading) / 2

size_math_df = math_pass.groupby(['School Size'])
size_read_df = read_pass.groupby(['School Size'])
size_math_stu_count = size_math_df["Student ID"].count()
size_read_stu_count = size_read_df["Student ID"].count()
size_percent_passing_math = size_math_stu_count/size_stu_count
size_percent_passing_reading = size_read_stu_count/size_stu_count
size_overall_passing_rate = (size_percent_passing_math + size_percent_passing_reading) / 2

type_math_df = math_pass.groupby(['type'])
type_read_df = read_pass.groupby(['type'])
type_math_stu_count = type_math_df["Student ID"].count()
type_read_stu_count = type_read_df["Student ID"].count()
type_percent_passing_math = type_math_stu_count/type_stu_count
type_percent_passing_reading = type_read_stu_count/type_stu_count
type_overall_passing_rate = (type_percent_passing_math + type_percent_passing_reading) / 2

In [70]:
# scores by school spending
spend_summary_table = pd.DataFrame({"Total Students": spend_stu_count,
                                    "Average Math Score": spend_math,
                                    "Average Reading Score": spend_read,
                                    "Percent Passing Math": spend_percent_passing_math,
                                    "Percent Passing Reading": spend_percent_passing_reading,
                                    "Overall Passing Rate": spend_overall_passing_rate})

spend_summary_table.head()

,Total Students,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Spending Summary,,,,,,
$585,6368,83.363065,83.964039,0.937029,0.966866,0.951947
$585-615,2723,83.529196,83.838414,0.941241,0.958869,0.950055
$615-645,17766,78.061635,81.434088,0.714004,0.836148,0.775076
$645-675,12313,77.049297,81.005604,0.662308,0.811094,0.736701


In [71]:
# scores by school size
size_summary_table = pd.DataFrame({"Total Students": size_stu_count,
                                    "Average Math Score": size_math,
                                    "Average Reading Score": size_read,
                                    "Percent Passing Math": size_percent_passing_math,
                                    "Percent Passing Reading": size_percent_passing_reading,
                                    "Overall Passing Rate": size_overall_passing_rate})

size_summary_table.head()

,Total Students,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Size,,,,,,
Small (<1000),1389,83.828654,83.974082,0.939525,0.960403,0.949964
Medium (1000-2000),8522,83.372682,83.867989,0.936165,0.967731,0.951948
Large (2000-5000),29259,77.477597,81.198674,0.686524,0.821252,0.753888


In [72]:
# scores by school type
type_summary_table = pd.DataFrame({"Total Students": type_stu_count,
                                    "Average Math Score": type_math,
                                    "Average Reading Score": type_read,
                                    "Percent Passing Math": type_percent_passing_math,
                                    "Percent Passing Reading": type_percent_passing_reading,
                                    "Overall Passing Rate": type_overall_passing_rate})

type_summary_table.head()

,Total Students,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
type,,,,,,
Charter,12194,83.406183,83.902821,0.937018,0.966459,0.951739
District,26976,76.987026,80.962485,0.665184,0.809052,0.737118


In [75]:
bin_join.describe()

,Student ID,reading_score,math_score,School ID,size,budget,Budget Bins
count,39170.000000,39170.00000,39170.000000,39170.000000,39170.000000,3.917000e+04,39170.000000
mean,19584.500000,81.87784,78.985371,6.978172,3332.957110,2.117241e+06,629.293541
std,11307.549359,10.23958,12.309968,4.444329,1323.914069,8.749987e+05,25.034815
min,0.000000,63.00000,55.000000,0.000000,427.000000,2.480870e+05,578.000000
25%,9792.250000,73.00000,69.000000,3.000000,1858.000000,1.081356e+06,625.000000
50%,19584.500000,82.00000,79.000000,7.000000,2949.000000,1.910635e+06,638.000000
75%,29376.750000,91.00000,89.000000,11.000000,4635.000000,3.022020e+06,650.000000
max,39169.000000,99.00000,99.000000,14.000000,4976.000000,3.124928e+06,655.000000
